# Chat Models với Claude - Hướng dẫn chi tiết

Notebook này cung cấp hướng dẫn toàn diện về cách sử dụng `ChatAnthropic` trong LangChain để tương tác với các mô hình Claude.

## Nội dung chính:
1. Giới thiệu về Chat Models
2. Các mô hình Claude khả dụng
3. Tham số cấu hình quan trọng
4. Các loại tin nhắn (Messages)
5. Phương thức invoke và stream
6. Các use case nâng cao

## 1. Setup môi trường

In [ ]:
# Import các thư viện cần thiết
import os
from dotenv import load_dotenv
from langchain_anthropic import ChatAnthropic
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

# Load environment variables
load_dotenv(dotenv_path='../../.env')

# Kiểm tra API key
api_key = os.getenv('ANTHROPIC_API_KEY')
if not api_key or api_key == 'your-api-key-here':
    raise ValueError("Vui lòng cấu hình ANTHROPIC_API_KEY trong file .env")

print("✅ Setup thành công!")

## 2. Giới thiệu về Chat Models

### Chat Models là gì?
Chat Models là các mô hình ngôn ngữ được thiết kế đặc biệt cho các cuộc hội thoại. Khác với Text Models truyền thống:
- **Input**: Danh sách các messages (tin nhắn) thay vì text đơn thuần
- **Output**: AIMessage object chứa response có cấu trúc
- **Context**: Duy trì ngữ cảnh hội thoại tốt hơn

## 3. Các mô hình Claude khả dụng

Anthropic cung cấp 3 dòng model Claude 3 với các đặc điểm khác nhau:

In [ ]:
# Danh sách các model Claude 3 và đặc điểm
claude_models = {
    "claude-3-opus-20240229": {
        "description": "Mạnh nhất, phù hợp cho task phức tạp",
        "context_window": 200000,
        "cost": "Cao nhất",
        "use_cases": ["Phân tích phức tạp", "Creative writing", "Code generation nâng cao"]
    },
    "claude-3-sonnet-20240229": {
        "description": "Cân bằng giữa hiệu năng và chi phí",
        "context_window": 200000,
        "cost": "Trung bình",
        "use_cases": ["Chatbot", "Tóm tắt văn bản", "Code review"]
    },
    "claude-3-haiku-20240307": {
        "description": "Nhanh và tiết kiệm chi phí",
        "context_window": 200000,
        "cost": "Thấp nhất",
        "use_cases": ["Classification", "Q&A đơn giản", "Data extraction"]
    }
}

# Hiển thị thông tin các model
for model_name, info in claude_models.items():
    print(f"\n🤖 {model_name}")
    print(f"   📝 {info['description']}")
    print(f"   📊 Context window: {info['context_window']:,} tokens")
    print(f"   💰 Chi phí: {info['cost']}")
    print(f"   🎯 Use cases: {', '.join(info['use_cases'])}")

## 4. Tham số cấu hình quan trọng

Khi khởi tạo ChatAnthropic, có nhiều tham số quan trọng cần hiểu:

In [ ]:
# Ví dụ về các tham số khác nhau
# 1. Model với temperature cao (creative)
creative_model = ChatAnthropic(
    model="claude-3-sonnet-20240229",
    temperature=0.9,  # 0-1: càng cao càng creative/random
    max_tokens=500,   # Giới hạn độ dài output
    anthropic_api_key=api_key
)

# 2. Model với temperature thấp (deterministic)
precise_model = ChatAnthropic(
    model="claude-3-sonnet-20240229",
    temperature=0.1,  # Output ổn định, ít biến đổi
    max_tokens=500,
    anthropic_api_key=api_key
)

# 3. Model với cấu hình đầy đủ
full_config_model = ChatAnthropic(
    model="claude-3-sonnet-20240229",
    temperature=0.7,
    max_tokens=1000,
    top_p=0.95,       # Nucleus sampling
    top_k=40,         # Top-k sampling
    timeout=30,       # Timeout trong giây
    max_retries=2,    # Số lần retry khi lỗi
    anthropic_api_key=api_key
)

print("✅ Đã khởi tạo 3 model với cấu hình khác nhau")

### So sánh output với temperature khác nhau

In [ ]:
# Test cùng một prompt với temperature khác nhau
test_prompt = "Viết một câu mở đầu cho câu chuyện về AI"

print("🎨 Creative Model (temp=0.9):")
creative_response = creative_model.invoke(test_prompt)
print(creative_response.content)

print("\n🎯 Precise Model (temp=0.1):")
precise_response = precise_model.invoke(test_prompt)
print(precise_response.content)

## 5. Các loại tin nhắn (Message Types)

LangChain hỗ trợ nhiều loại message khác nhau:

In [ ]:
# Import thêm các message types
from langchain_core.messages import (
    SystemMessage,
    HumanMessage,
    AIMessage,
    FunctionMessage,
    ToolMessage
)

# Khởi tạo model cho ví dụ
chat = ChatAnthropic(
    model="claude-3-sonnet-20240229",
    temperature=0.7,
    anthropic_api_key=api_key
)

### 5.1 SystemMessage - Định nghĩa vai trò và hành vi

In [ ]:
# Ví dụ 1: AI như một chuyên gia
messages = [
    SystemMessage(content="""Bạn là một chuyên gia về Machine Learning với 10 năm kinh nghiệm.
    Hãy trả lời các câu hỏi một cách chuyên nghiệp, sử dụng thuật ngữ kỹ thuật khi cần thiết,
    và luôn đưa ra ví dụ cụ thể."""),
    HumanMessage(content="Gradient Descent là gì?")
]

response = chat.invoke(messages)
print("🎓 Expert Response:")
print(response.content)

In [ ]:
# Ví dụ 2: AI giải thích cho người mới bắt đầu
messages = [
    SystemMessage(content="""Bạn là một giáo viên kiên nhẫn, giải thích mọi thứ một cách
    đơn giản nhất có thể. Tránh sử dụng thuật ngữ phức tạp và luôn dùng ví dụ từ cuộc sống hàng ngày."""),
    HumanMessage(content="Gradient Descent là gì?")
]

response = chat.invoke(messages)
print("👶 Beginner-Friendly Response:")
print(response.content)

### 5.2 Chuỗi hội thoại với nhiều messages

In [ ]:
# Xây dựng một cuộc hội thoại đầy đủ
conversation = [
    SystemMessage(content="Bạn là trợ lý AI hữu ích, luôn nhớ ngữ cảnh cuộc trò chuyện."),
    HumanMessage(content="Tôi đang học Python. Bạn có thể gợi ý một dự án cho người mới bắt đầu không?"),
    AIMessage(content="Tôi gợi ý bạn làm một ứng dụng To-Do List đơn giản. Đây là dự án tuyệt vời cho người mới bắt đầu vì nó bao gồm các khái niệm cơ bản như: input/output, lưu trữ dữ liệu, và xử lý logic."),
    HumanMessage(content="Nghe hay đấy! Tôi nên bắt đầu từ đâu?")
]

# Gửi toàn bộ conversation
response = chat.invoke(conversation)
print("💬 Conversation Response:")
print(response.content)

## 6. Phương thức Invoke vs Stream

### 6.1 Invoke - Nhận response đầy đủ

In [ ]:
import time

# Đo thời gian với invoke
start_time = time.time()

response = chat.invoke("Giải thích về Neural Networks trong 3 câu")

end_time = time.time()
print(f"⏱️ Thời gian response: {end_time - start_time:.2f} giây\n")
print("📝 Response:")
print(response.content)
print(f"\n📊 Metadata:")
print(f"- ID: {response.id}")
print(f"- Model: {response.response_metadata.get('model', 'N/A')}")
print(f"- Usage: {response.response_metadata.get('usage', 'N/A')}")

### 6.2 Stream - Nhận response theo từng phần

In [ ]:
# Stream cho trải nghiệm real-time
print("🌊 Streaming Response:")
print("-" * 50)

stream_start = time.time()
first_token_time = None
token_count = 0

for chunk in chat.stream("Viết một đoạn văn ngắn về tương lai của AI trong giáo dục"):
    if first_token_time is None:
        first_token_time = time.time()
    
    print(chunk.content, end="", flush=True)
    token_count += 1

stream_end = time.time()

print(f"\n\n📊 Stream Statistics:")
print(f"- Time to first token: {first_token_time - stream_start:.2f}s")
print(f"- Total time: {stream_end - stream_start:.2f}s")
print(f"- Chunks received: {token_count}")

### 6.3 Async Operations

In [ ]:
import asyncio

# Async invoke
async def async_chat_example():
    # Gọi nhiều requests đồng thời
    prompts = [
        "Định nghĩa Machine Learning trong 1 câu",
        "Định nghĩa Deep Learning trong 1 câu",
        "Định nghĩa AI trong 1 câu"
    ]
    
    # Tạo tasks
    tasks = [chat.ainvoke(prompt) for prompt in prompts]
    
    # Chờ tất cả hoàn thành
    start = time.time()
    responses = await asyncio.gather(*tasks)
    end = time.time()
    
    print(f"⚡ Async execution time: {end - start:.2f}s\n")
    
    for prompt, response in zip(prompts, responses):
        print(f"❓ {prompt}")
        print(f"✅ {response.content}\n")

# Chạy async function
await async_chat_example()

## 7. Use Cases Nâng Cao

### 7.1 Structured Output với Response Format

In [ ]:
# Yêu cầu output có cấu trúc
structured_prompt = """Phân tích câu sau và trả về kết quả theo format JSON:
Câu: "LangChain là một framework mạnh mẽ cho việc phát triển ứng dụng AI."

Format mong muốn:
{
    "sentiment": "positive/negative/neutral",
    "key_entities": [list of important entities],
    "main_topic": "topic of the sentence",
    "confidence": 0.0-1.0
}"""

response = chat.invoke(structured_prompt)
print("📋 Structured Output:")
print(response.content)

# Parse JSON response
import json
try:
    parsed_response = json.loads(response.content)
    print("\n✅ Parsed successfully:")
    for key, value in parsed_response.items():
        print(f"  - {key}: {value}")
except:
    print("\n⚠️ Response không phải JSON hợp lệ")

### 7.2 Chain of Thought Prompting

In [ ]:
# Chain of Thought để giải quyết vấn đề phức tạp
cot_messages = [
    SystemMessage(content="""Bạn là một chuyên gia giải quyết vấn đề. 
    Khi được hỏi, hãy:
    1. Phân tích vấn đề từng bước
    2. Giải thích logic của mỗi bước
    3. Đưa ra kết luận cuối cùng"""),
    HumanMessage(content="""Một cửa hàng bán 3 loại sản phẩm:
    - Sản phẩm A: giá 100k, lợi nhuận 30%
    - Sản phẩm B: giá 200k, lợi nhuận 20%  
    - Sản phẩm C: giá 150k, lợi nhuận 25%
    
    Nếu bán được 10A, 5B, và 8C trong ngày, tổng lợi nhuận là bao nhiêu?""")
]

response = chat.invoke(cot_messages)
print("🧠 Chain of Thought Response:")
print(response.content)

### 7.3 Multi-turn Conversation với Context Management

In [ ]:
# Quản lý context trong hội thoại dài
class ConversationManager:
    def __init__(self, chat_model, max_history=10):
        self.chat = chat_model
        self.history = []
        self.max_history = max_history
        self.system_message = SystemMessage(
            content="Bạn là trợ lý AI thông minh, luôn nhớ context cuộc trò chuyện."
        )
    
    def add_message(self, role, content):
        if role == "human":
            self.history.append(HumanMessage(content=content))
        elif role == "ai":
            self.history.append(AIMessage(content=content))
        
        # Giới hạn history
        if len(self.history) > self.max_history:
            self.history = self.history[-self.max_history:]
    
    def get_response(self, user_input):
        # Thêm user input
        self.add_message("human", user_input)
        
        # Tạo messages list với system message
        messages = [self.system_message] + self.history
        
        # Gọi model
        response = self.chat.invoke(messages)
        
        # Lưu response
        self.add_message("ai", response.content)
        
        return response.content

# Demo conversation manager
conv_manager = ConversationManager(chat)

# Cuộc hội thoại mẫu
conversation_flow = [
    "Xin chào! Tôi muốn học về Python.",
    "Tôi đã biết các kiến thức cơ bản rồi. Bước tiếp theo là gì?",
    "OOP nghe phức tạp quá. Bạn có thể cho ví dụ đơn giản không?"
]

for i, user_input in enumerate(conversation_flow, 1):
    print(f"\n👤 Turn {i} - User: {user_input}")
    response = conv_manager.get_response(user_input)
    print(f"🤖 Claude: {response}")

## 8. Best Practices và Tips

### 8.1 Error Handling

In [ ]:
from langchain_core.exceptions import OutputParserException
from anthropic import RateLimitError, APIError

def safe_chat_invoke(chat_model, messages, max_retries=3):
    """Invoke chat với error handling đầy đủ"""
    for attempt in range(max_retries):
        try:
            response = chat_model.invoke(messages)
            return response
        
        except RateLimitError as e:
            print(f"⚠️ Rate limit reached. Waiting 60s... (Attempt {attempt + 1}/{max_retries})")
            if attempt < max_retries - 1:
                time.sleep(60)
            else:
                raise e
        
        except APIError as e:
            print(f"❌ API Error: {e}")
            if attempt < max_retries - 1:
                time.sleep(5)
            else:
                raise e
        
        except Exception as e:
            print(f"❌ Unexpected error: {e}")
            raise e
    
    return None

# Test error handling
test_messages = [HumanMessage(content="Hello Claude!")]
safe_response = safe_chat_invoke(chat, test_messages)
if safe_response:
    print("✅ Safe invoke successful:", safe_response.content)

### 8.2 Token Counting và Cost Estimation

In [ ]:
# Ước tính token và chi phí
def estimate_tokens(text):
    """Ước tính số token (rough estimate)"""
    # Quy tắc thô: ~1 token = 4 ký tự cho tiếng Anh
    # Tiếng Việt có thể khác
    return len(text) / 4

def estimate_cost(input_tokens, output_tokens, model="claude-3-sonnet-20240229"):
    """Ước tính chi phí (USD)"""
    # Giá tham khảo (có thể thay đổi)
    pricing = {
        "claude-3-opus-20240229": {"input": 0.015, "output": 0.075},
        "claude-3-sonnet-20240229": {"input": 0.003, "output": 0.015},
        "claude-3-haiku-20240307": {"input": 0.00025, "output": 0.00125}
    }
    
    if model in pricing:
        input_cost = (input_tokens / 1000) * pricing[model]["input"]
        output_cost = (output_tokens / 1000) * pricing[model]["output"]
        return input_cost + output_cost
    return 0

# Demo
test_input = "Giải thích về Machine Learning"
response = chat.invoke(test_input)

input_tokens = estimate_tokens(test_input)
output_tokens = estimate_tokens(response.content)

print(f"📊 Token Usage Estimate:")
print(f"- Input tokens: ~{input_tokens:.0f}")
print(f"- Output tokens: ~{output_tokens:.0f}")
print(f"- Total tokens: ~{input_tokens + output_tokens:.0f}")

cost = estimate_cost(input_tokens, output_tokens)
print(f"\n💰 Estimated cost: ${cost:.6f} USD")

## 9. Tổng kết

Trong notebook này, chúng ta đã học về:

1. **Chat Models** - Hiểu rõ về chat models và cách chúng hoạt động
2. **Các model Claude** - So sánh Opus, Sonnet, và Haiku
3. **Tham số cấu hình** - Temperature, max_tokens, và các tham số khác
4. **Message Types** - SystemMessage, HumanMessage, AIMessage
5. **Invoke vs Stream** - Khi nào dùng phương thức nào
6. **Async operations** - Xử lý đồng thời nhiều requests
7. **Advanced patterns** - Structured output, CoT, conversation management
8. **Best practices** - Error handling, token counting, cost estimation

### Next Steps:
- Thử nghiệm với các model khác nhau
- Xây dựng chatbot hoàn chỉnh
- Tích hợp với vector stores cho RAG
- Implement streaming trong web applications

In [ ]:
# Quick reference card
print("🎯 CLAUDE CHAT MODELS - QUICK REFERENCE\n")
print("📌 Basic Usage:")
print("   chat = ChatAnthropic(model='claude-3-sonnet-20240229', temperature=0.7)")
print("   response = chat.invoke('Your prompt here')\n")

print("📌 With Messages:")
print("   messages = [")
print("       SystemMessage(content='You are a helpful assistant'),")
print("       HumanMessage(content='Hello!')")
print("   ]")
print("   response = chat.invoke(messages)\n")

print("📌 Streaming:")
print("   for chunk in chat.stream('Your prompt'):")
print("       print(chunk.content, end='')\n")

print("📌 Key Parameters:")
print("   - temperature: 0.0-1.0 (creativity)")
print("   - max_tokens: output length limit")
print("   - model: opus/sonnet/haiku variants")